## Customer Segmentation Analysis with RFM analysis and KMeans Clustering

In [5]:
import pandas as pd
import matplotlib as plt
from sklearn.cluster import KMeans

In [ ]:
# Loading the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
data_frame = pd.read_excel(url)

In [26]:

data_frame.head()
data_frame.shape

(541909, 8)